In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import plotly.express as px
from sklearn.manifold import TSNE

In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


In [ ]:
# Flatten images for input layer
x_train = x_train.reshape((len(x_train), 784))
x_test = x_test.reshape((len(x_test), 784))

In [ ]:
# Define the autoencoder architecture
input_img = Input(shape=(784,))


In [ ]:
# Encoder
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(3, activation='linear')(encoded)  # Bottleneck (3D latent space)


In [ ]:
# Decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)


In [ ]:
# Build models
autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)


In [ ]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 0.3445 - val_loss: 0.2082
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 0.2034 - val_loss: 0.1918
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - loss: 0.1888 - val_loss: 0.1826
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.1812 - val_loss: 0.1773
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.1759 - val_loss: 0.1741
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.1729 - val_loss: 0.1720
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.1710 - val_loss: 0.1705
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.1695 - val_loss: 0.1694
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1685 - val_loss: 0.1682
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.1674 - val_loss: 0.1673
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.1661 - val_loss: 0.1664
Epoch 12/50
235/235 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Encode test data
num_samples = 1000
encoded_imgs = encoder.predict(x_test[:num_samples])
labels = y_test[:num_samples]

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
# Apply t-SNE to reduce high-dimensional data to 3D
tsne = TSNE(n_components=3, random_state=42)
tsne_results = tsne.fit_transform(encoded_imgs)


In [ ]:
# Create an 3D scatter plot with Plotly
fig = px.scatter_3d(
    x=tsne_results[:, 0],
    y=tsne_results[:, 1],
    z=tsne_results[:, 2],
    color=labels.astype(str),
    labels={'x': 't-SNE Dimension 1', 'y': 't-SNE Dimension 2', 'z': 't-SNE Dimension 3'},
    title='Interactive 3D t-SNE Visualization of Encoded MNIST Images',
    opacity=0.8
)

fig.update_traces(marker=dict(size=5, opacity=0.8, line=dict(width=0.5)))

# Show plot
fig.show()
